<a href="https://colab.research.google.com/github/L4riss4/Python/blob/main/TP_04_Projeto_de_bloco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TP 04 - PROJETO DE BLOCO

Realizando consultas com Pandas e exportando dados no formato JSON

Dados lidos a partir de arquivos CSV gerados a partir de um banco de dados SQL


###Importando tabelas e realizando as adequações de datas


In [ ]:
#Importando bibliotecas
import pandas as pd
import json
from tabulate import tabulate
from datetime import datetime, timedelta

In [ ]:
#Importando dados de arquivos CSV
df_cargos = pd.read_csv('tabela_cargos.csv', sep=';', encoding='utf -8')
df_departamentos = pd.read_csv('tabela_departamentos.csv', sep=';', encoding='utf-8')
df_dependentes = pd.read_csv('tabela_dependentes.csv', sep=';', encoding='latin-1')
df_funcionarios = pd.read_csv('tabela_funcionarios.csv', sep=';', encoding='utf -8')
df_historicosalarios = pd.read_csv('tabela_historicosalarios.csv', sep=';', encoding='utf -8')
df_projetosdesenvolvidos = pd.read_csv('tabela_projetosdesenvolvidos.csv', sep=';', encoding='utf -8')
df_recursosprojetos = pd.read_csv('tabela_recursosprojetos.csv', sep=';', encoding='utf -8')


###Realizando Consultas
Consultas baseadas no arquivo de avaliação TP 04 de Projeto de Bloco

Trazer a média dos salários (atual) dos funcionários responsáveis por projetos concluídos, agrupados por departamento.

In [ ]:
#Integrando tabelas
df_projetofuncionario = pd.merge(df_funcionarios, df_projetosdesenvolvidos, left_on='FuncionarioID', right_on='FuncionarioResponsavel')
df_projfuncdepart = pd.merge(df_projetofuncionario, df_departamentos, on='DepartamentoID')

#Apenas projetos concluídos
df_projfuncdepart = df_projfuncdepart[df_projfuncdepart['StatusProj'] == 'Concluído']

#Média salarial
df_mediasalarial = df_projfuncdepart.groupby('NomeDepartamento')['Salario'].mean().reset_index()
df_mediasalarial

,NomeDepartamento,Salario
0,RH,3950.0
1,TI,6000.0
2,Vendas,3800.0


 Identificar os três recursos materiais mais usados nos projetos, listando a descrição do recurso
e a quantidade total usada.

In [ ]:
#Selecionando o tipo de recurso
df_recursosutilizados = df_recursosprojetos[df_recursosprojetos['TipoRecurso']=='Material']

#Somando a quantidade
df_recursosutilizados = df_recursosutilizados.groupby('DescricaoRecurso')['QuantidadeUtilizada'].sum()

#Selecionando os 3 com maiores quantias
df_recursosutilizados = df_recursosutilizados.sort_values(ascending=False).head(3)
df_recursosutilizados = df_recursosutilizados.reset_index()
df_recursosutilizados

,DescricaoRecurso,QuantidadeUtilizada
0,Estações de Trabalho,9
1,Componentes de Segurança,6
2,Hardware de Servidor,4


Calcular o custo total dos projetos por departamento, considerando apenas os projetos
'Concluídos'.

In [ ]:
#Concatenação de dataframes funcionarios, departamentos e projetos
df_departamentofuncionario = pd.merge(df_funcionarios, df_departamentos, on='DepartamentoID')
df_projetodepartamento = pd.merge(df_projetosdesenvolvidos, df_departamentofuncionario, left_on='FuncionarioResponsavel', right_on='FuncionarioID')

#Filtrando pelos projetos com status concluído
df_custodepartamento = df_projetodepartamento[df_projetodepartamento['StatusProj'] == 'Concluído']

#Agrupando pelo departamento e somando os custos
df_custodepartamento = df_custodepartamento.groupby('NomeDepartamento')['CustoProjeto'].sum().reset_index()
df_custodepartamento

,NomeDepartamento,CustoProjeto
0,RH,60000
1,TI,150000
2,Vendas,20000


 Listar todos os projetos com seus respectivos nomes, custo, data de início, data de conclusão e o nome do funcionário responsável, que estejam 'Em Execução'.

In [ ]:
#Concatenção de dataframes projetos e funcionarios
df_projetofuncionario = pd.merge(df_projetosdesenvolvidos, df_funcionarios, left_on='FuncionarioResponsavel', right_on='FuncionarioID')

#Filtrando projetos em execução
df_projetofuncionario = df_projetofuncionario[df_projetofuncionario['StatusProj'] == 'Em Execução']

#Colunas a serem exibidas
df_row = ['NomeProjeto', 'CustoProjeto', 'DataInicio', 'DataConclusao', 'NomeFuncionario', 'StatusProj']
df_projetofuncionario = df_projetofuncionario[df_row]
df_projetofuncionario

,NomeProjeto,CustoProjeto,DataInicio,DataConclusao,NomeFuncionario,StatusProj
2,Aplicativo Mobile,50000,01/03/2024,01/09/2024,Carlos Oliveira,Em Execução
3,Migração de Dados,20000,01/04/2024,01/10/2024,Ana Souza,Em Execução
5,Plataforma E-commerce,60000,05/02/2024,05/09/2024,Patricia Souza,Em Execução
6,Sistema de CRM,45000,15/01/2024,05/08/2024,Isabela Ferreira,Em Execução
7,Integração de API,10000,15/03/2024,15/10/2024,Gustavo Lima,Em Execução


Identificar o projeto com o maior número de dependentes envolvidos, considerando que os dependentes são associados aos funcionários que estão gerenciando os projetos.

In [ ]:
#Concatenação de dataframes dependentes e projetos
df_projdependentes = pd.merge(df_dependentes, df_projetosdesenvolvidos, left_on='FuncionarioID', right_on='FuncionarioResponsavel')

#Agrupando por projeto e realizando contagem de depentes associados
df_projdependentes = df_projdependentes.groupby('NomeProjeto')['DependenteID'].count().reset_index()

#Colunas a serem exibidas
df_projdependentes.columns = ['NomeProjeto','QuantidadeDependentes']

#Ordenando por ordem decrescente
df_projdependentes = df_projdependentes.sort_values(by='QuantidadeDependentes', ascending=False)
df_projdependentes

,NomeProjeto,QuantidadeDependentes
0,Aplicativo Mobile,2
2,Automação de RH,2
4,Migração de Dados,2
6,Sistema ERP,2
8,Treinamento de TI,2
9,Website Corporativo,2
1,Atualização de ERP,1
3,Integração de API,1
5,Plataforma E-commerce,1
7,Sistema de CRM,1


Conversão de df em arquivos JSON

In [ ]:
def arq_json(nome_arquivo, obj_json):
  if not nome_arquivo.endswith('.json'):
    nome_arquivo += '.json'

  if isinstance(obj_json, pd.DataFrame):
    obj_json = obj_json.to_dict(orient='records')

  with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
    json.dump(obj_json, arquivo, indent=4)
  print(f"\nArquivo {nome_arquivo} salvo com sucesso")

In [ ]:
arq_json('DF_RECURSOS_PROJETOS', df_recursosutilizados)
arq_json('DF_RESPONSAVEL_PROJETO', df_projetofuncionario)
arq_json('DF_CUSTO_PROJETO', df_custodepartamento)


Arquivo DF_RECURSOS_PROJETOS.json salvo com sucesso

Arquivo DF_RESPONSAVEL_PROJETO.json salvo com sucesso

Arquivo DF_CUSTO_PROJETO.json salvo com sucesso
